## Setup
* Use yolo v5 medium with dropout 0.25

In [1]:
GLOBAL_SEED = 42

import os
os.environ['PYTHONHASHSEED'] = str(GLOBAL_SEED)

import random as rnd
import numpy as np
from numpy import random as np_rnd
import pandas as pd
import pickle

import torch
from ultralytics import YOLO

c:\Users\USER\AppData\Local\Programs\Python\Python37\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def seed_everything(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    # python random
    rnd.seed(seed)
    # numpy random
    np_rnd.seed(seed)
    # RAPIDS random
    try:
        cupy.random.seed(seed)
    except:
        pass
    # tf random
    try:
        tf_rnd.set_seed(seed)
    except:
        pass
    # pytorch random
    try:
        torch.backends.cudnn.benchmark = False
        torch.backends.cudnn.deterministic = True
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    except:
        pass

def pickleIO(obj, src, op="r"):
    if op=="w":
        with open(src, op + "b") as f:
            pickle.dump(obj, f)
    elif op=="r":
        with open(src, op + "b") as f:
            tmp = pickle.load(f)
        return tmp
    else:
        print("unknown operation")
        return obj
    
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print('Error: Creating directory. ' + directory)

def findIdx(data_x, col_names):
    return [int(i) for i, j in enumerate(data_x) if j in col_names]

def diff(first, second):
    second = set(second)
    return [item for item in first if item not in second]

In [3]:
class CFG:
    debug = False
    model_name = "yolov5mu"

## Define Model & Training

In [4]:
# Load the model
model = YOLO('./yolo_models/yolov5mu.pt')  # load a pretrained model

In [5]:
# Perform inference
model.train(
    data='yolov5_custom_cfg.yaml', dropout=0.25, imgsz=640, epochs=30, warmup_epochs=0, patience=10, batch=4,
    optimizer="AdamW", lr0=5e-4, lrf=1e-7, momentum=0.9, weight_decay=1e-4, cos_lr=True,
    save_period=1, exist_ok=True, device=0
)

New https://pypi.org/project/ultralytics/8.0.146 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.145  Python-3.7.9 torch-1.13.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3070 Ti Laptop GPU, 8192MiB)
WARNING  Upgrade to torch>=2.0.0 for deterministic training.
engine\trainer: task=detect, mode=train, model=./yolo_models/yolov5mu.pt, data=yolov5_custom_cfg.yaml, epochs=30, patience=10, batch=4, imgsz=640, save=True, save_period=1, cache=False, device=0, workers=8, project=None, name=None, exist_ok=True, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.25, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_str